In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ast
from pycocotools.coco import COCO
import skimage.io as io
import glob
import os

In [ ]:
json_path = '/root/data/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/body_parts_annotations.json'
example_coco = COCO(json_path)
# df = pd.read_csv('./dataset_world_coordinates.csv')
df = pd.read_csv('./dataset_complete.csv')

In [ ]:
# df = df.dropna()

In [ ]:
all_images_path = glob.glob('/root/data/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/*/*.jpg')
local_dic = {}
for path in all_images_path:
    if 'rectified' not in path:
        local_dic[os.path.basename(path)] = path

In [ ]:
# get random row
randind = np.random.randint(0, df.shape[0])
print(randind)
random_row = df.iloc[randind, :]

In [ ]:
# category_ids = example_coco.getCatIds()
category_ids = [1, 3, 4, 5, 6, 7, 8]
image_ids = random_row['left_image_key']
image_data = example_coco.loadImgs([image_ids])[0]

# load and display instance annotations
file_name = image_data['coco_url'].split('%2F')[2].split('?alt')[0]
image = io.imread(local_dic[file_name])
f ,ax = plt.subplots(1, figsize=(20, 20))
ax.imshow(image); 
# ax.axis('off')
# plt.axis('off')
# pylab.rcParams['figure.figsize'] = (8.0, 10.0)
annotation_ids = example_coco.getAnnIds(imgIds=[image_data['id']], catIds=category_ids, iscrowd=None)
annotations = example_coco.loadAnns(annotation_ids)
example_coco.showAnns(annotations)
for ann in annotations:
    bbox = ann['bbox']
    rec = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], 
                            edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
    ax.add_patch(rec)
    # ax.text(bbox[0], bbox[1]-10, id2class[str(ann['category_id'])], fontsize=16)
plt.show()
# plt.savefig('test.jpg')

In [ ]:
coordinates = np.array([ast.literal_eval(col) for col in random_row[1:-2]])

In [ ]:
from utils import minimum_bounding_rectangle

In [ ]:
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

In [ ]:
poly = minimum_bounding_rectangle(np.stack([coordinates[:, 0], coordinates[:, 2]]).transpose())
f, ax = plt.subplots(1)
ax.scatter(coordinates[:, 0], coordinates[:, 2])
for (x, y, s) in zip(coordinates[:, 0], coordinates[:, 2], list(df.columns)[1:-2]):
    ax.text(x, y+0.005, s)
p = patches.Polygon(poly, edgecolor='r', facecolor='none')
ax.add_patch(p)
plt.show()

In [ ]:
XY_poly_area = PolyArea(poly[:, 0], poly[:, 1])
print("xy area : {} m^2".format(XY_poly_area))

In [ ]:
poly = minimum_bounding_rectangle(np.stack([coordinates[:, 2], coordinates[:, 1]]).transpose())
f, ax = plt.subplots(1)
ax.scatter(coordinates[:, 2], coordinates[:, 1])
for (x, y, s) in zip(coordinates[:, 2], coordinates[:, 1], list(df.columns)[1:-2]):
    ax.text(x, y+0.005, s)
p = patches.Polygon(poly, edgecolor='r', facecolor='none')
ax.add_patch(p)
plt.show()

In [ ]:
width = coordinates[:, 1].max() - coordinates[:, 1].min()
print("Thickness: {} m".format(width))

In [ ]:
total_volume = XY_poly_area*width

In [ ]:
total_volume

# all 

In [ ]:
dfna = df.dropna()

In [ ]:
dfna.shape

In [ ]:
volumes = []
for rowind in range(dfna.shape[0]):
    row = dfna.iloc[rowind, 1:-2]
    
    coordinates = np.array([ast.literal_eval(col) for col in row])
    poly = minimum_bounding_rectangle(np.stack([coordinates[:, 0], coordinates[:, 2]]).transpose())
    XY_poly_area = PolyArea(poly[:, 0], poly[:, 1])
    width = coordinates[:, 1].max() - coordinates[:, 1].min()
    volume = XY_poly_area*width
    volumes.append(volume)
    # print("xy area : {} m^2".format(XY_poly_area))

In [ ]:
plt.scatter(volumes, dfna['ground_truth'])
plt.xlabel('Predicted Volume')
plt.ylabel('Weight')
plt.xlim([0, 0.1])
plt.show()

# back to length

In [ ]:
ground_truth_weights = {'181001010001':4315,
'181001010002':5981,
'181001010003':2773,
'181001010004':3164,
'181001010005':4480,
'181001010006':2559,
'181001010007':2121,
'181001010008':3949,
'181001010009':1957,
'181001010010':2997}

In [ ]:
ground_truth_lengths = {'181001010001': 0.695,
 '1810010100010': 0.655,
 '181001010002': 0.75,
 '181001010003': 0.585,
 '181001010004': 0.625,
 '181001010005': 0.685,
 '181001010006': 0.645,
 '181001010007': 0.535,
 '181001010008': 0.66,
 '181001010009': 0.56,
 '181001010010': 0.655}

In [ ]:
w2l = {}
for (k,v) in ground_truth_weights.items():
    w2l[v] = ground_truth_lengths[k]

In [ ]:
gt_length = [w2l[w] for w in list(df.iloc[:, -2])]

In [ ]:
plt.scatter(list(df.iloc[:, 2]), gt_length)
plt.ylabel('ground truth length')
plt.xlabel('predicted length')
plt.ylim([0.6, 0.7])
plt.show()